# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 27 2022 3:18PM,247420,10,Enova-10711,Emerginnova,Released
1,Sep 27 2022 3:18PM,247420,10,DNMD-24106,Emerginnova,Released
2,Sep 27 2022 3:14PM,247418,10,0086006777,ISDIN Corporation,Released
3,Sep 27 2022 3:14PM,247418,10,0086006849,ISDIN Corporation,Released
4,Sep 27 2022 3:14PM,247418,10,0086006857,ISDIN Corporation,Released
5,Sep 27 2022 3:14PM,247418,10,0086006858,ISDIN Corporation,Released
6,Sep 27 2022 3:14PM,247418,10,0086006903,ISDIN Corporation,Released
7,Sep 27 2022 3:14PM,247418,10,0086006910,ISDIN Corporation,Released
8,Sep 27 2022 3:14PM,247418,10,0086006980,ISDIN Corporation,Released
9,Sep 27 2022 3:14PM,247418,10,0086015764,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,247414,Released,2
26,247415,Released,16
27,247417,Released,5
28,247418,Released,10
29,247420,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247414,NaN,NaN,2.0
247415,NaN,NaN,16.0
247417,NaN,NaN,5.0
247418,NaN,NaN,10.0
247420,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247202,0.0,0.0,1.0
247205,0.0,1.0,0.0
247331,1.0,0.0,2.0
247332,0.0,0.0,1.0
247345,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247202,0,0,1
247205,0,1,0
247331,1,0,2
247332,0,0,1
247345,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247202,0,0,1
1,247205,0,1,0
2,247331,1,0,2
3,247332,0,0,1
4,247345,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247202,,,1
1,247205,,1,
2,247331,1,,2
3,247332,,,1
4,247345,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 27 2022 3:18PM,247420,10,Emerginnova
2,Sep 27 2022 3:14PM,247418,10,ISDIN Corporation
12,Sep 27 2022 3:12PM,247417,10,Bio-PRF
17,Sep 27 2022 3:12PM,247415,10,"Methapharm, Inc."
33,Sep 27 2022 3:11PM,247414,10,"Nextsource Biotechnology, LLC"
35,Sep 27 2022 3:03PM,247413,22,"NBTY Global, Inc."
36,Sep 27 2022 2:51PM,247411,10,"Citieffe, Inc."
37,Sep 27 2022 2:26PM,247410,19,"AdvaGen Pharma, Ltd"
67,Sep 27 2022 2:22PM,247409,10,Emerginnova
68,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 27 2022 3:18PM,247420,10,Emerginnova,,,2
1,Sep 27 2022 3:14PM,247418,10,ISDIN Corporation,,,10
2,Sep 27 2022 3:12PM,247417,10,Bio-PRF,,,5
3,Sep 27 2022 3:12PM,247415,10,"Methapharm, Inc.",,,16
4,Sep 27 2022 3:11PM,247414,10,"Nextsource Biotechnology, LLC",,,2
5,Sep 27 2022 3:03PM,247413,22,"NBTY Global, Inc.",,,1
6,Sep 27 2022 2:51PM,247411,10,"Citieffe, Inc.",,,1
7,Sep 27 2022 2:26PM,247410,19,"AdvaGen Pharma, Ltd",,11,19
8,Sep 27 2022 2:22PM,247409,10,Emerginnova,,,1
9,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 3:18PM,247420,10,Emerginnova,2,,
1,Sep 27 2022 3:14PM,247418,10,ISDIN Corporation,10,,
2,Sep 27 2022 3:12PM,247417,10,Bio-PRF,5,,
3,Sep 27 2022 3:12PM,247415,10,"Methapharm, Inc.",16,,
4,Sep 27 2022 3:11PM,247414,10,"Nextsource Biotechnology, LLC",2,,
5,Sep 27 2022 3:03PM,247413,22,"NBTY Global, Inc.",1,,
6,Sep 27 2022 2:51PM,247411,10,"Citieffe, Inc.",1,,
7,Sep 27 2022 2:26PM,247410,19,"AdvaGen Pharma, Ltd",19,11,
8,Sep 27 2022 2:22PM,247409,10,Emerginnova,1,,
9,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 3:18PM,247420,10,Emerginnova,2,,
1,Sep 27 2022 3:14PM,247418,10,ISDIN Corporation,10,,
2,Sep 27 2022 3:12PM,247417,10,Bio-PRF,5,,
3,Sep 27 2022 3:12PM,247415,10,"Methapharm, Inc.",16,,
4,Sep 27 2022 3:11PM,247414,10,"Nextsource Biotechnology, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 3:18PM,247420,10,Emerginnova,2.0,NaN,NaN
1,Sep 27 2022 3:14PM,247418,10,ISDIN Corporation,10.0,NaN,NaN
2,Sep 27 2022 3:12PM,247417,10,Bio-PRF,5.0,NaN,NaN
3,Sep 27 2022 3:12PM,247415,10,"Methapharm, Inc.",16.0,NaN,NaN
4,Sep 27 2022 3:11PM,247414,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 3:18PM,247420,10,Emerginnova,2.0,0.0,0.0
1,Sep 27 2022 3:14PM,247418,10,ISDIN Corporation,10.0,0.0,0.0
2,Sep 27 2022 3:12PM,247417,10,Bio-PRF,5.0,0.0,0.0
3,Sep 27 2022 3:12PM,247415,10,"Methapharm, Inc.",16.0,0.0,0.0
4,Sep 27 2022 3:11PM,247414,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3216109,104.0,3.0,2.0
15,247352,7.0,47.0,13.0
19,247410,19.0,11.0,0.0
20,742174,36.0,17.0,0.0
21,494407,1.0,1.0,0.0
22,742228,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3216109,104.0,3.0,2.0
1,15,247352,7.0,47.0,13.0
2,19,247410,19.0,11.0,0.0
3,20,742174,36.0,17.0,0.0
4,21,494407,1.0,1.0,0.0
5,22,742228,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,104.0,3.0,2.0
1,15,7.0,47.0,13.0
2,19,19.0,11.0,0.0
3,20,36.0,17.0,0.0
4,21,1.0,1.0,0.0
5,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,15,Released,7.0
2,19,Released,19.0
3,20,Released,36.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,2.0,13.0,0.0,0.0,0.0,0.0
Executing,3.0,47.0,11.0,17.0,1.0,0.0
Released,104.0,7.0,19.0,36.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,2.0,13.0,0.0,0.0,0.0,0.0
1,Executing,3.0,47.0,11.0,17.0,1.0,0.0
2,Released,104.0,7.0,19.0,36.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,2.0,13.0,0.0,0.0,0.0,0.0
1,Executing,3.0,47.0,11.0,17.0,1.0,0.0
2,Released,104.0,7.0,19.0,36.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()